# Data Cleaning

Data cleaning has been separated into Python and R. The data gathered using Python and the Twitter API have been cleaned using Python and libraries such as NLTK and CountVectorizer. The data gathered directly from sources, as well as data gathered via API calls in R have been cleaned using R. The specific codes for cleaning the gathered data can be found below.

## Tweets

In [ ]:
## imports
import pandas as pd
import numpy as np
import re
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer

## read in tweets
raw_tweets = pd.read_json('../../data/00-raw-data/tweets.json')

## filter for only english tweets and the text body
n = len(raw_tweets.keys()) - 1
tweets_l = []
for i in range(0, n):
    if(raw_tweets[str(i)]['lang'] == 'en'):
        tweets_l.append(raw_tweets[str(i)]['text'])
tweets_l = list(dict.fromkeys(tweets_l))

## filter out unwanted chars
tweets_printable = []

for text in tweets_l:
    new_text=""
    for character in text:
        if character in string.printable:
            new_text+=character
    tweets_printable.append(new_text)

initial_clean = []

## cleaning out URLs
for tweet in tweets_printable:
    clean = re.sub(r"@[A-Za-z0-9_]+", "", tweet)
    clean = re.sub(r'http\S+', "", clean)
    clean = re.sub(r'https\S+', "", clean)
    clean = re.sub(r'www\S+', "", clean)
    clean = clean.strip()
    initial_clean.append(clean)

## gather hashtags
tweets_hashtags = []
for tweet in initial_clean:
    hashtags = re.findall("#([a-zA-Z0-9_]{1,50})", tweet)
    if hashtags:
        tweets_hashtags.append(hashtags)

flat_list = [hashtag for sublist in tweets_hashtags for hashtag in sublist]

## remove stopwords
stop = nltk.corpus.stopwords.words('english')

tokenized = [nltk.tokenize.word_tokenize(tweet.lower().strip()) for tweet in flat_list]
no_stopwords = []

for tweet in tokenized:
    for word in tweet:
        if word not in stop:
            no_stopwords.append(word)

# ref: https://stackoverflow.com/questions/10017147/removing-a-list-of-characters-in-string
to_remove = [".",",","!","?",":",";","_"]
cleaned_tweets = [tweet.translate({ord(x): '' for x in to_remove}) for tweet in no_stopwords]

## vectorize
vectorizer=CountVectorizer()   

Xs  =  vectorizer.fit_transform(cleaned_tweets)   
tweets = pd.DataFrame.from_dict(vectorizer.vocabulary_, orient='index')
tweets.reset_index(inplace=True)
tweets.columns = ['Word', 'Count']

tweets.replace('', np.nan, inplace=True)
tweets.dropna(inplace=True)
tweets.sort_values('Count', ascending=False, inplace=True)
tweets.reset_index(inplace=True)

## remove incomplete / nonsense words
to_drop = [21, 23, 32, 40, 43, 48, 71, 119, 171, 178 , 208, 213, 217, 257, 260]
tweets.drop(to_drop,axis=0, inplace=True)

## export
tweets.to_csv('./../../data/01-modified-data/Tweets_final.csv')

## BLS

### Employment and Wages

In [ ]:
## read in file
df = pd.read_excel('../../data/00-raw-data/wages_(by_occupation_may_2021).xlsx')

## select specific columns + reformat df
df = df[['OCC_TITLE', 'O_GROUP', 'TOT_EMP', 'EMP_PRSE', 'A_MEAN', 'MEAN_PRSE']]
df = df.iloc[1:, :]
df['Target'] = 'X'
df['Target_Num'] = 0
majors = df[df['O_GROUP'] == 'major']['OCC_TITLE']

## create string and numeric representations for major occupation titles
df.loc[1:73, 'Target'] = majors[1]
df.loc[1:73, 'Target_Num'] = 1
df.loc[74:131, 'Target'] = majors[74]
df.loc[74:131, 'Target_Num'] = 2
df.loc[132:167, 'Target'] = majors[132]
df.loc[132:167, 'Target_Num'] = 3
df.loc[168:228, 'Target'] = majors[168]
df.loc[168:228, 'Target_Num'] = 4
df.loc[229:307, 'Target'] = majors[229]
df.loc[229:307, 'Target_Num'] = 5
df.loc[308:333, 'Target'] = majors[308]
df.loc[308:333, 'Target_Num'] = 6
df.loc[334:348, 'Target'] = majors[334]
df.loc[334:348, 'Target_Num'] = 7
df.loc[349:445, 'Target'] = majors[349]
df.loc[349:445, 'Target_Num'] = 8
df.loc[446:507, 'Target'] = majors[446]
df.loc[446:507, 'Target_Num'] = 9
df.loc[508:609, 'Target'] = majors[508]
df.loc[508:609, 'Target_Num'] = 10
df.loc[610:636, 'Target'] = majors[610]
df.loc[610:636, 'Target_Num'] = 11
df.loc[637:679, 'Target'] = majors[637]
df.loc[637:679, 'Target_Num'] = 12
df.loc[680:712, 'Target'] = majors[680]
df.loc[680:712, 'Target_Num'] = 13
df.loc[713:730, 'Target'] = majors[713]
df.loc[713:730, 'Target_Num'] = 14
df.loc[731:790, 'Target'] = majors[731]
df.loc[731:790, 'Target_Num'] = 15
df.loc[791:833, 'Target'] = majors[791]
df.loc[791:833, 'Target_Num'] = 16
df.loc[834:942, 'Target'] = majors[834]
df.loc[834:942, 'Target_Num'] = 17
df.loc[943:966, 'Target'] = majors[943]
df.loc[943:966, 'Target_Num'] = 18
df.loc[967:1069, 'Target'] = majors[967]
df.loc[967:1069, 'Target_Num'] = 19
df.loc[1070:1144, 'Target'] = majors[1070]
df.loc[1070:1144, 'Target_Num'] = 20
df.loc[1145:1311, 'Target'] = majors[1145]
df.loc[1145:1311, 'Target_Num'] = 21
df.loc[1312:1402, 'Target'] = majors[1312]
df.loc[1312:1402, 'Target_Num'] = 22

## remove rows containing '*'
df = df[df.apply(lambda x: (~x.astype(str).str.contains('\*', case=True, regex=True)))].dropna()

## export
df.to_csv('./../../data/01-modified-data/occupations_detailed_(employment_and_wage).csv')

### Employment (by sex) and Wages

In [ ]:
## read in files
df1 = pd.read_csv('../../data/01-modified-data/occupations_detailed_(employment_and_wage).csv')
df2 = pd.read_excel('../../data/00-raw-data/employment_(by_occupation_and_by_sex).xlsx')

## drop unneeded column
df1.drop(df1.iloc[:, [0]], axis=1, inplace=True)

## keep specific columns
df1 = df1[['OCC_TITLE', 'O_GROUP', 'TOT_EMP', 'A_MEAN', 'Target', 'Target_Num']]

## convert occupation titles to lowercase
df1['OCC_TITLE'] = df1['OCC_TITLE'].str.lower()

## keep specific columns/rows
df2 = df2.iloc[8:,:3]

## rename columns
df2.columns.values[0] = 'Occupation'
df2.columns.values[1] = 'Total Employed'
df2.columns.values[2] = 'Women (%)'

## convert Women (%) column to numeric
df2 = df2[pd.to_numeric(df2['Women (%)'], errors='coerce').notnull()]

df2 = df2.reset_index(drop=True)

## create new column
df2['Men (%)'] = 100 - df2.loc[:]['Women (%)']

## change occupation column to lowercase
df2['Occupation'] = df2['Occupation'].str.lower()

## merge dfs
df = df1.merge(df2, how='inner', left_on='OCC_TITLE', right_on='Occupation')

## filter for detailed occupation groups
df = df[df['O_GROUP'] == 'detailed']

df.reset_index(drop=True, inplace=True)

## drop unneeded columns
df.drop(df.iloc[:, [1,6]], axis=1, inplace=True)

## export
df.to_csv('./../../data/01-modified-data/occupations_detailed_(employment_by_sex_and_wage).csv')

### Earnings and Unemployment Rates (by Educational Attainment)

```{r}
df <- read.xlsx('../../data/00-raw-data/earnings_and_unemployment_rates_(by_educational_attaiment).xlsx', 1, header=F)
## remove unnecessary rows
df <- df %>% filter(row_number() %in% c(2:10))
## create column names
names(df) <- df[1,]
df <- df[-1,] %>%
        mutate(`Median usual weekly earnings` = as.numeric(`Median usual weekly earnings`),
               `Unemployment rate` = as.numeric(`Unemployment rate`)) %>%
        rename(`Median weekly earnings ($)` = `Median usual weekly earnings`,
               `Unemployment rate (%)` = `Unemployment rate`)
## adjusting units for this item
df[1, 3] <- df[1, 3] * 100
## convert to long format
df <- df %>% gather("Measure", "Value", 2:3)
write.csv(df, "earnings_and_unemployment_rates_(by_educational_attainment)_final.csv", row.names=F)
```


### Employment (by Occupation and by Sex)

```{r}
df <- read.xlsx('../../data/00-raw-data/employment_(by_occupation_and_by_sex).xlsx', 1, header=F)
## create column names
df[7,1] <- "Occupation"
df[7,2] <- "Total Count"
df[7,3] <- "Women"
df[7,4] <- "Men"
## remove unnecessary rows and columns
df <- df %>% filter(row_number() %in% c(7:602)) %>% 
  select(c(X1, X2, X3, X4))
names(df) <- df[1,]
df <- df[-1,] %>%
        mutate(`Total Count` = as.numeric(`Total Count`) * 1000,
               Women = as.numeric(Women), Men = 100 - Women)
## keep major titles
df <- df %>% filter(row_number() %in% c(1, 4, 38, 69, 88, 113, 141, 158, 
                                         165, 178, 210, 257, 258, 273, 294, 
                                         307, 316, 337, 338, 357, 412, 413, 
                                         422, 460, 497, 498, 562)) %>%
        gather("Measure", "Value", 2:4)
## create column for sex to adjust for wide to long format change
df$sex <- c(rep('B', 27), rep('F', 27), rep('M', 27))
df <- df %>% mutate(Measure = replace(Measure, Measure=='Total Count', 'Count'),
                    Measure = replace(Measure, Measure=='Women', 'Employment (%)'),
                    Measure = replace(Measure, Measure=='Men', 'Employment (%)'))
write.csv(df, "employment_(by_occupation_and_by_sex)_final.csv", row.names=F)
```


### Hours Worked (by Sex and by Occupation)

```{r}
df <- read.xlsx('../../data/00-raw-data/hours_worked_(by_sex_and_by_occupation).xlsx', 1, header=F)
## remove unnecessary rows and columns
df <- df %>% filter(row_number() %in% c(9:50)) %>% 
        select(c(X1, X2, X3, X7, X8, X9)) %>%
        filter(row_number() %in% c(1, 2, 5, 6, 9, 12, 
                                   15, 16, 19, 20, 23, 26,
                                   29, 30, 33, 34, 37, 40)) %>% 
        mutate(X2 = as.numeric(X2) * 1000, X3 = as.numeric(X3) * 1000, 
               X7 = as.numeric(X7) * 1000, X8 = as.numeric(X8),
               X9 = as.numeric(X9))
## create column names
df[1,1] <- 'Total'
df[7,1] <- 'Total'
df[13,1] <- 'Total'
## create separate DFs for total, male, and female data
df_t <- df[1:6,]
df_t <- df_t %>%
        rename(Category = X1, `No. people at work` = X2, 
               `No. people who worked < 35hrs` = X3, 
               `No. people who worked 35+ hrs` = X7, 
               `Average hrs worked among all workers` = X8, 
               `Average hrs worked among full-time workers` = X9)
df_t$sex <- "B"
df_m <- df[7:12,]
df_m <- df_m %>%
        rename(Category = X1, `No. people at work` = X2, 
               `No. people who worked < 35hrs` = X3, 
               `No. people who worked 35+ hrs` = X7, 
               `Average hrs worked among all workers` = X8, 
               `Average hrs worked among full-time workers` = X9)
df_m$sex <- "M"
df_f <- df[13:18,]
df_f <- df_f %>%
        rename(Category = X1, `No. people at work` = X2, 
               `No. people who worked < 35hrs` = X3, 
               `No. people who worked 35+ hrs` = X7, 
               `Average hrs worked among all workers` = X8, 
               `Average hrs worked among full-time workers` = X9)
df_f$sex <- "F"
## merge DFs and change to long format
df <- bind_rows(df_m, df_f, df_t) %>% gather("Measure", "Value", 2:6)
## renaming specific rows
df$Category[df$Category == "Natural resources, construction, and maintenance occupations(1)"] <- "Natural resources, construction, and maintenance occupations"
write.csv(df, "hours_worked_(by_sex_and_by_occupation)_final.csv", row.names=F)
```


### Wages (by Occupation) from May 2021

```{r}
df <- read.xlsx('../../data/00-raw-data/wages_(by_occupation_may_2021).xlsx', 1, header=F)
## remove unnecessary columns
df <- df[, c(10, 11, 12, 13, 19, 20)]
## create column names
names(df) <- df[1,]
df <- df[-1,]
## keep major titles
## removing NAs since empty data in this case is useless
df <- df[df$O_GROUP=='major',] %>% 
        select(-c(O_GROUP, EMP_PRSE, MEAN_PRSE)) %>%
        mutate(TOT_EMP = as.numeric(TOT_EMP), 
               A_MEAN = as.numeric(A_MEAN)) %>%
        rename(Occupation = OCC_TITLE, `Total Employment` = TOT_EMP,
               `Mean Annual Wage` = A_MEAN) %>%
        gather("Measure", "Value", 2:3)
write.csv(df, "wages_(by_occupation_may_2021)_final.csv", row.names=F)
```


### Employment Rates

```{r}
df <- read.csv('../../data/00-raw-data/employmentRate.csv')
## rename columns
df <- df %>% 
        rename(`Total Employment (%)` = LNS12300000, 
               `Employment among men (%)` = LNS12300001, 
               `Employment among women (%)` = LNS12300002) %>%
        select(5:8)

df1 <- read.csv('../../data/00-raw-data/employmentRate16to24.csv')
df1 <- df1 %>% 
        rename(`Employment among men 16-24 (%)` = LNS12324885, 
               `Employment among women 16-24 (%)` = LNS12324886) %>%
        select(5:7)

df2 <- read.csv('../../data/00-raw-data/employmentRate25to34.csv')
df2 <- df2 %>% 
        rename(`Employment among men 25-34 (%)` = LNS12300164, 
               `Employment among women 25-34 (%)` = LNS12300327) %>%
        select(5:7)

df3 <- read.csv('../../data/00-raw-data/employmentRate35to44.csv')
df3 <- df3 %>% 
        rename(`Employment among men 35-44 (%)` = LNS12300173, 
               `Employment among women 35-44 (%)` = LNS12300334) %>%
        select(5:7)

df4 <- read.csv('../../data/00-raw-data/employmentRate45to54.csv')
df4 <- df4 %>% 
        rename(`Employment among men 45-54 (%)` = LNS12300182, 
               `Employment among women 45-54 (%)` = LNS12300341) %>%
        select(5:7)

df5 <- read.csv('../../data/00-raw-data/employmentRate55+.csv')
df5 <- df5 %>% 
        rename(`Employment among men 55+ (%)` = LNS12324231, 
               `Employment among women 55+ (%)` = LNS12324232) %>%
        select(5:7)
## merge DFs and change to long format
df <- bind_cols(df, df1[,1:2], df2[,1:2], df3[,1:2], df4[,1:2], df5[,1:2]) %>%
        relocate(date, .before=1) %>% gather("Measure", "Value", 2:14) 
df$sex <- c(rep('B', 896), rep('M', 896), rep('F', 896), 
            rep('M', 896), rep('F', 896), 
            rep('M', 896), rep('F', 896), 
            rep('M', 896), rep('F', 896), 
            rep('M', 896), rep('F', 896), 
            rep('M', 896), rep('F', 896))
df <- df %>% 
        mutate(Measure = replace(Measure, Measure=='Employment among men (%)', 'Total Employment (%)'),
               Measure = replace(Measure, Measure=='Employment among women (%)', 'Total Employment (%)'),
               Measure = replace(Measure, Measure=='Employment among men 16-24 (%)', 'Employment 16-24 (%)'),
               Measure = replace(Measure, Measure=='Employment among women 16-24 (%)', 'Employment 16-24 (%)'),
               Measure = replace(Measure, Measure=='Employment among men 25-34 (%)', 'Employment 25-34 (%)'),
               Measure = replace(Measure, Measure=='Employment among women 25-34 (%)', 'Employment 25-34 (%)'),
               Measure = replace(Measure, Measure=='Employment among men 35-44 (%)', 'Employment 35-44 (%)'),
               Measure = replace(Measure, Measure=='Employment among women 35-44 (%)', 'Employment 35-44 (%)'),
               Measure = replace(Measure, Measure=='Employment among men 45-54 (%)', 'Employment 45-54 (%)'),
               Measure = replace(Measure, Measure=='Employment among women 45-54 (%)', 'Employment 45-54 (%)'),
               Measure = replace(Measure, Measure=='Employment among men 55+ (%)', 'Employment 55+ (%)'),
               Measure = replace(Measure, Measure=='Employment among women 55+ (%)', 'Employment 55+ (%)'),
               date = as.Date(date, format="%Y-%m-%d"))
write.csv(df, "employmentRates_final.csv", row.names=F)
```


## Center for American Women and Politics

```{r}
## create Year DF for merging later. 1965-2021 chosen b/c
## 1965 = min Year among all DFs, and 2021 = max Year among all DFs
Year <- data.frame(Year = c(1965:2021))

df <- read.xlsx('../../data/00-raw-data/percent_us_women_governors.xlsx', 1, header=F)
df <- df %>% filter(row_number() %in% c(3:50))
names(df) <- df[1,]
df <- df[-1,] %>%
        mutate(Year = as.numeric(Year),
               `Share of state governors who are women` = as.numeric(`Share of state governors who are women`) * 100) %>%
        rename(`Female state governors (%)` = `Share of state governors who are women`)

df1 <- read.xlsx('../../data/00-raw-data/percent_us_women_house_rep.xlsx', 1, header=F)
df1 <- df1 %>% filter(row_number() %in% c(3:32))
names(df1) <- df1[1,]
df1 <- df1[-1,] %>% 
        mutate(`Starting date of congressional term` = as.numeric(`Starting date of congressional term`),
               `Share of U.S. representatives who are women` = as.numeric(`Share of U.S. representatives who are women`) * 100) %>%
        rename(Year = `Starting date of congressional term`,
               `Female U.S. representatives (%)` = `Share of U.S. representatives who are women`)

df2 <- read.xlsx('../../data/00-raw-data/percent_us_women_senators.xlsx', 1, header=F)
df2 <- df2 %>% filter(row_number() %in% c(3:32))
names(df2) <- df2[1,]
df2 <- df2[-1,] %>% 
        mutate(`Starting date of congressional term` = as.numeric(`Starting date of congressional term`),
               `Share of U.S. senators who are women` = as.numeric(`Share of U.S. senators who are women`) * 100) %>%
        rename(Year = `Starting date of congressional term`,
               `Female U.S. senators (%)` = `Share of U.S. senators who are women`) %>%
        select(-`NA`)

df3 <- read.xlsx('../../data/00-raw-data/percent_us_women_state_legislators.xlsx', 1, header=F)
df3 <- df3 %>% filter(row_number() %in% c(3:41))
names(df3) <- df3[1,]
df3 <- df3[-1,] %>% 
        mutate(Year = as.numeric(Year),
               `Share of state legislators who are women` = as.numeric(`Share of state legislators who are women`) * 100) %>%
        rename(`Female U.S. state legislators (%)` = `Share of state legislators who are women`)

## join dfs
df <- full_join(Year, df, by='Year') 
df <- full_join(df, df1, by='Year')
df <- full_join(df, df2, by='Year')
df <- full_join(df, df3, by='Year')
## convert to long format
df <- df %>% gather("Measure", "Value", 2:5) %>% drop_na()
write.csv(df, "percent_us_women_in_gov_final.csv", row.names=F)
```


## Census Bureau

### % of U.S. Population w/ a College Degree (by Gender) from 1940-2021

```{r}
df <- read.xlsx('../../data/00-raw-data/Percentage-of-the-us-population-with-a-college-degree-by-gender-1940-2021.xlsx', 2, header=F)
## create column names
df[3,1] <- "Year"
df[3,4] <- "drop"
## remove unnecessary rows
df <- df %>% filter(row_number() %in% c(3:68))
names(df) <- df[1,]
df <- df[-1,] %>% 
        mutate(Year = as.numeric(Year), Male = as.numeric(Male), Female = as.numeric(Female)) %>%
        rename(`Men with a college degree in the U.S. (%)` = Male,
               `Women with a college degree in the U.S. (%)` = Female) %>%
        select(-drop) %>% 
        gather("Measure", "Value", 2:3) ## convert to long format
write.csv(df, "Percentage-of-the-us-population-with-a-college-degree-by-gender-1940-2021.csv", row.names=F)
```


### School Completion (by Age and Sex) from 1940-2021

```{r}
df <- read.xlsx('../../data/00-raw-data/School_Completion_(by_Age_and_Sex)_from_1940_to_2021.xlsx', 1, header=F)
## create column names
df[7,1] <- "Year"
df[7,2] <- "Total"
df[7,3] <- "0-4 years"
df[7,4] <- "5-8 years"
df[7,5] <- "9-11 years"
df[7,6] <- "12 years"
df[7,7] <- "13-15 years"
df[7,8] <- "16+ years"
## remove unnecessary rows and columns
df <- df %>% select(c(X1:X8)) %>% filter(!row_number() %in% c(1:6))
names(df) <- df[1,]
df <- df[-1,]
df1 <- df %>% filter(row_number() %in% c(268:401))
df1_m <- df1 %>% filter(row_number() %in% c(2:67)) %>%
        rename(`Total (25-34, M)` = Total, 
               `0-4 years (25-34, M)` = `0-4 years`,
               `5-8 years (25-34, M)` = `5-8 years`,
               `9-11 years (25-34, M)` = `9-11 years`,
               `12 years (25-34, M)` = `12 years`,
               `13-15 years (25-34, M)` = `13-15 years`,
               `16+ years (25-34, M)` = `16+ years`)
df1_f <- df1 %>% filter(row_number() %in% c(69:134)) %>%
        rename(`Total (25-34, F)` = Total, 
               `0-4 years (25-34, F)` = `0-4 years`,
               `5-8 years (25-34, F)` = `5-8 years`,
               `9-11 years (25-34, F)` = `9-11 years`,
               `12 years (25-34, F)` = `12 years`,
               `13-15 years (25-34, F)` = `13-15 years`,
               `16+ years (25-34, F)` = `16+ years`)
df2 <- df %>% filter(row_number() %in% c(469:602))
df2_m <- df2 %>% filter(row_number() %in% c(2:67)) %>%
        rename(`Total (35-54, M)` = Total, 
               `0-4 years (35-54, M)` = `0-4 years`,
               `5-8 years (35-54, M)` = `5-8 years`,
               `9-11 years (35-54, M)` = `9-11 years`,
               `12 years (35-54, M)` = `12 years`,
               `13-15 years (35-54, M)` = `13-15 years`,
               `16+ years (35-54, M)` = `16+ years`)
df2_f <- df2 %>% filter(row_number() %in% c(69:134)) %>%
        rename(`Total (35-54, F)` = Total, 
               `0-4 years (35-54, F)` = `0-4 years`,
               `5-8 years (35-54, F)` = `5-8 years`,
               `9-11 years (35-54, F)` = `9-11 years`,
               `12 years (35-54, F)` = `12 years`,
               `13-15 years (35-54, F)` = `13-15 years`,
               `16+ years (35-54, F)` = `16+ years`)
df3 <- df %>% filter(row_number() %in% c(670:803))
df3_m <- df3 %>% filter(row_number() %in% c(2:67)) %>%
        rename(`Total (55+, M)` = Total, 
               `0-4 years (55+, M)` = `0-4 years`,
               `5-8 years (55+, M)` = `5-8 years`,
               `9-11 years (55+, M)` = `9-11 years`,
               `12 years (55+, M)` = `12 years`,
               `13-15 years (55+, M)` = `13-15 years`,
               `16+ years (55+, M)` = `16+ years`)
df3_f <- df3 %>% filter(row_number() %in% c(69:134)) %>%
        rename(`Total (55+, F)` = Total, 
               `0-4 years (55+, F)` = `0-4 years`,
               `5-8 years (55+, F)` = `5-8 years`,
               `9-11 years (55+, F)` = `9-11 years`,
               `12 years (55+, F)` = `12 years`,
               `13-15 years (55+, F)` = `13-15 years`,
               `16+ years (55+, F)` = `16+ years`)
## merge DFs
df <- cbind(df1_m, df1_f[, 2:8], 
            df2_m[, 2:8], df2_f[, 2:8], 
            df3_m[, 2:8], df3_f[, 2:8]) %>% 
        relocate(c(9, 16, 23, 30, 37), .after=2) %>%
        relocate(c(14, 20, 26, 32, 38), .after=8) %>%
        relocate(c(19, 24, 29, 34, 39), .after=14) %>%
        relocate(c(24, 28, 32, 36, 40), .after=20) %>%
        relocate(c(29, 32, 35, 38, 41), .after=26) %>%
        relocate(c(34, 36, 38, 40, 42), .after=32) %>%
        relocate(c(39:43), .after=38) %>% gather("Measure", "Value", 2:43) 
## create column for sex to adjust for wide to long format change
df$sex <- c(rep('M', 66), rep('F', 66),
            rep('M', 66), rep('F', 66),
            rep('M', 66), rep('F', 66),
            rep('M', 66), rep('F', 66),
            rep('M', 66), rep('F', 66),
            rep('M', 66), rep('F', 66),
            rep('M', 66), rep('F', 66),
            rep('M', 66), rep('F', 66),
            rep('M', 66), rep('F', 66),
            rep('M', 66), rep('F', 66),
            rep('M', 66), rep('F', 66),
            rep('M', 66), rep('F', 66),
            rep('M', 66), rep('F', 66),
            rep('M', 66), rep('F', 66),
            rep('M', 66), rep('F', 66),
            rep('M', 66), rep('F', 66),
            rep('M', 66), rep('F', 66),
            rep('M', 66), rep('F', 66),
            rep('M', 66), rep('F', 66),
            rep('M', 66), rep('F', 66),
            rep('M', 66), rep('F', 66))
## renaming rows
df <- df %>% 
        mutate(Measure = replace(Measure, Measure=='Total (25-34, M)', 'Total Count (25-34)'),
               Measure = replace(Measure, Measure=='Total (25-34, F)', 'Total Count (25-34)'),
               Measure = replace(Measure, Measure=='Total (35-54, M)', 'Total Count (35-54)'),
               Measure = replace(Measure, Measure=='Total (35-54, F)', 'Total Count (35-54)'),
               Measure = replace(Measure, Measure=='Total (55+, M)', 'Total Count (55+)'),
               Measure = replace(Measure, Measure=='Total (55+, F)', 'Total Count (55+)'),
               Measure = replace(Measure, Measure=='0-4 years (25-34, M)', 'No. of People w/ 0-4 years of education (25-34)'),
               Measure = replace(Measure, Measure=='0-4 years (25-34, F)', 'No. of People w/ 0-4 years of education (25-34)'),
               Measure = replace(Measure, Measure=='0-4 years (35-54, M)', 'No. of People w/ 0-4 years of education (35-54)'),
               Measure = replace(Measure, Measure=='0-4 years (35-54, F)', 'No. of People w/ 0-4 years of education (35-54)'),
               Measure = replace(Measure, Measure=='0-4 years (55+, M)', 'No. of People w/ 0-4 years of education (55+)'),
               Measure = replace(Measure, Measure=='0-4 years (55+, F)', 'No. of People w/ 0-4 years of education (55+)'),
               Measure = replace(Measure, Measure=='5-8 years (25-34, M)', 'No. of People w/ 5-8 years of education (25-34)'),
               Measure = replace(Measure, Measure=='5-8 years (25-34, F)', 'No. of People w/ 5-8 years of education (25-34)'),
               Measure = replace(Measure, Measure=='5-8 years (35-54, M)', 'No. of People w/ 5-8 years of education (35-54)'),
               Measure = replace(Measure, Measure=='5-8 years (35-54, F)', 'No. of People w/ 5-8 years of education (35-54)'),
               Measure = replace(Measure, Measure=='5-8 years (55+, M)', 'No. of People w/ 5-8 years of education (55+)'),
               Measure = replace(Measure, Measure=='5-8 years (55+, F)', 'No. of People w/ 5-8 years of education (55+)'),
               Measure = replace(Measure, Measure=='9-11 years (25-34, M)', 'No. of People w/ 9-11 years of education (25-34)'),
               Measure = replace(Measure, Measure=='9-11 years (25-34, F)', 'No. of People w/ 9-11 years of education (25-34)'),
               Measure = replace(Measure, Measure=='9-11 years (35-54, M)', 'No. of People w/ 9-11 years of education (35-54)'),
               Measure = replace(Measure, Measure=='9-11 years (35-54, F)', 'No. of People w/ 9-11 years of education (35-54)'),
               Measure = replace(Measure, Measure=='9-11 years (55+, M)', 'No. of People w/ 9-11 years of education (55+)'),
               Measure = replace(Measure, Measure=='9-11 years (55+, F)', 'No. of People w/ 9-11 years of education (55+)'),
               Measure = replace(Measure, Measure=='12 years (25-34, M)', 'No. of People w/ 12 years of education (25-34)'),
               Measure = replace(Measure, Measure=='12 years (25-34, F)', 'No. of People w/ 12 years of education (25-34)'),
               Measure = replace(Measure, Measure=='12 years (35-54, M)', 'No. of People w/ 12 years of education (35-54)'),
               Measure = replace(Measure, Measure=='12 years (35-54, F)', 'No. of People w/ 12 years of education (35-54)'),
               Measure = replace(Measure, Measure=='12 years (55+, M)', 'No. of People w/ 12 years of education (55+)'),
               Measure = replace(Measure, Measure=='12 years (55+, F)', 'No. of People w/ 12 years of education (55+)'),
               Measure = replace(Measure, Measure=='13-15 years (25-34, M)', 'No. of People w/ 13-15 years of education (25-34)'),
               Measure = replace(Measure, Measure=='13-15 years (25-34, F)', 'No. of People w/ 13-15 years of education (25-34)'),
               Measure = replace(Measure, Measure=='13-15 years (35-54, M)', 'No. of People w/ 13-15 years of education (35-54)'),
               Measure = replace(Measure, Measure=='13-15 years (35-54, F)', 'No. of People w/ 13-15 years of education (35-54)'),
               Measure = replace(Measure, Measure=='13-15 years (55+, M)', 'No. of People w/ 13-15 years of education (55+)'),
               Measure = replace(Measure, Measure=='13-15 years (55+, F)', 'No. of People w/ 13-15 years of education (55+)'),
               Measure = replace(Measure, Measure=='16+ years (25-34, M)', 'No. of People w/ 16+ years of education (25-34)'),
               Measure = replace(Measure, Measure=='16+ years (25-34, F)', 'No. of People w/ 16+ years of education (25-34)'),
               Measure = replace(Measure, Measure=='16+ years (35-54, M)', 'No. of People w/ 16+ years of education (35-54)'),
               Measure = replace(Measure, Measure=='16+ years (35-54, F)', 'No. of People w/ 16+ years of education (35-54)'),
               Measure = replace(Measure, Measure=='16+ years (55+, M)', 'No. of People w/ 16+ years of education (55+)'),
               Measure = replace(Measure, Measure=='16+ years (55+, F)', 'No. of People w/ 16+ years of education (55+)')) %>%
        mutate(Year = as.numeric(Year),
               Value = as.numeric(Value) * 1000)
write.csv(df, "School_Completion_(by_Age_and_Sex)_from_1940_to_2021_final.csv", row.names=F)
```


## National Center for Education Statistics

### Degrees (by Sex and by Field)

```{r}
df <- read.xlsx('../../data/00-raw-data/degrees_(by_sex_and_by_field).xls', 1, header=F)
## create column names
df[3,2] <- "Occupations"
df[3,3] <- "Bachelor's Total"
df[3,4] <- "Bachelor's Men"
df[3,5] <- "Bachelor's Women"
df[3,6] <- "Master's Total"
df[3,7] <- "Master's Men"
df[3,8] <- "Master's Women"
df[3,9] <- "Doctor's Total"
df[3,10] <- "Doctor's Men"
df[3,11] <- "Doctor's Women"
# removing all-encompassing major titles
df <- df %>% select(-X1) %>%
        filter(!row_number() %in% c(1, 2, 4, 6, 7, 64, 65, 87, 98, 138,
                                    223, 314, 326, 327, 351, 365, 394, 
                                    492, 493, 547, 548, 607, 617, 629,
                                    642, 676, 730, 932, 961, 982, 987,
                                    992, 1008, 1020, 1053, 1066, 1081, 
                                    1125, 1133, 1138, 1166, 1179, 1180,
                                    1212, 1221, 1239, 1249, 1314:1317))
names(df) <- df[1,]
df <- df[-1,] %>% mutate(`Bachelor's Total` = as.numeric(`Bachelor's Total`),
                         `Bachelor's Men` = as.numeric(`Bachelor's Men`),
                         `Bachelor's Women` = as.numeric(`Bachelor's Women`),
                         `Master's Total` = as.numeric(`Master's Total`),
                         `Master's Men` = as.numeric(`Master's Men`),
                         `Master's Women` = as.numeric(`Master's Women`),
                         `Doctor's Total` = as.numeric(`Doctor's Total`),
                         `Doctor's Men` = as.numeric(`Doctor's Men`),
                         `Doctor's Women` = as.numeric(`Doctor's Women`))
## remove leading and trailing whitespace
df$Occupations <- str_trim(df$Occupations) 
## convert to long format
df <- df %>% gather("Measure", "Value", 2:10) 
## create column for sex to adjust for wide to long format change
df$sex <- c(rep('B', 1265), rep('M', 1265), rep('F', 1265),
            rep('B', 1265), rep('M', 1265), rep('F', 1265),
            rep('B', 1265), rep('M', 1265), rep('F', 1265))
df <- df %>% 
        mutate(Measure = replace(Measure, Measure=="Bachelor's Total", "Bachelor's"),
               Measure = replace(Measure, Measure=="Bachelor's Men", "Bachelor's"),
               Measure = replace(Measure, Measure=="Bachelor's Women", "Bachelor's"),
               Measure = replace(Measure, Measure=="Master's Total", "Master's"),
               Measure = replace(Measure, Measure=="Master's Men", "Master's"),
               Measure = replace(Measure, Measure=="Master's Women", "Master's"),
               Measure = replace(Measure, Measure=="Doctor's Total", "Doctor's"),
               Measure = replace(Measure, Measure=="Doctor's Men", "Doctor's"),
               Measure = replace(Measure, Measure=="Doctor's Women", "Doctor's"))
write.csv(df, "degrees_(by_sex_and_by_field)_final.csv", row.names=F)
```


### Degrees (by Field) among Men

```{r}
df1 <- read.xlsx('../../data/00-raw-data/degrees_(by_field_males).xls', 1, header=F)
## create column names
df1[5,1] <- "Occupations"
df1[5,12] <- "Total (M)"
## remove unnecessary rows and columns
df1 <- df1 %>% select(c(X1, X12)) %>%
        filter(row_number() %in% c(5:40))
names(df1) <- df1[1,]
df1 <- df1[-1,]
## replace consecutive whitespaces with single space
df1$Occupations <- gsub("\\s+", " ", str_trim(df1$Occupations))
## remove special characters and numbers in specific rows
df1$Occupations[2] <- substr(df1$Occupations[2], 1, nchar(df1$Occupations[2]) - 3)
df1$Occupations[6] <- substr(df1$Occupations[6], 1, nchar(df1$Occupations[6]) - 3)
df1$Occupations[12] <- substr(df1$Occupations[12], 1, nchar(df1$Occupations[12]) - 3)

df2 <- read.xlsx('../../data/00-raw-data/degrees_(by_field_females).xls', 1, header=F)
## create column names
df2[5,1] <- "Occupations"
df2[5,12] <- "Total (F)"
## remove unnecessary rows and columns
df2 <- df2 %>% select(c(X1, X12)) %>%
        filter(row_number() %in% c(5:40))
names(df2) <- df2[1,]
df2 <- df2[-1,]
## replace consecutive whitespaces with single space
df2$Occupations <- gsub("\\s+", " ", str_trim(df2$Occupations))
## remove special characters and numbers in specific rows
df2$Occupations[2] <- substr(df2$Occupations[2], 1, nchar(df2$Occupations[2]) - 3)
df2$Occupations[6] <- substr(df2$Occupations[6], 1, nchar(df2$Occupations[6]) - 3)
df2$Occupations[12] <- substr(df2$Occupations[12], 1, nchar(df2$Occupations[12]) - 3)

## merge DFs
df <- cbind(df1, df2[,2]) %>%
        rename(`Total (F)` = `df2[, 2]`) %>% 
        mutate(`Total (M)` = as.numeric(`Total (M)`),
               `Total (F)` = as.numeric(`Total (F)`))
## convert to long format
df <- df %>% gather("Measure", "Value", 2:3) 
## create column for sex to adjust for wide to long format change
df$sex <- c(rep('M', 35), rep('F', 35))
df <- df %>% 
        mutate(Measure = replace(Measure, Measure=="Total (M)", "No. of People"),
               Measure = replace(Measure, Measure=="Total (F)", "No. of People"))
write.csv(df, "counts_(by_sex_and_by_field)_final.csv", row.names=F)
```


## Big Five Personality

```{r}
df <- read.csv('../../data/00-raw-data/big_five.csv', header=T)
## remove non-USA rows, reorder DF by sex (1 = M, 2 = F)
df <- df %>% filter(country == 'USA') %>% arrange(sex)
write.csv(df, "big_five_final.csv", row.names=F)
```